# Explaining the Agent Core Behavior

This notebook explains the apparent contradiction in the agent's behavior when it comes to task assessment versus actual response generation.

## Setup and Imports

First, let's import the necessary modules from the agentic core package.

In [1]:
import sys
import os
import logging

# Add the parent directory to sys.path to allow importing the agentic module
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..')))

# Import the core components
from agentic.core import AgentCore, AgentConfig
from agentic.core.tools import Tool, ToolResult

# Configure logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

## Creating a Calculator Tool

Let's create the same calculator tool from our example.

In [2]:
class CalculatorTool(Tool):
    """A simple calculator tool for demonstration purposes."""
    
    def __init__(self):
        """Initialize the calculator tool."""
        super().__init__(
            name="calculator",
            description="Performs basic arithmetic operations (add, subtract, multiply, divide)"
        )
    
    def execute(self, operation: str, a: float, b: float) -> ToolResult:
        """Execute the calculator operation."""
        try:
            if operation == "add":
                result = a + b
            elif operation == "subtract":
                result = a - b
            elif operation == "multiply":
                result = a * b
            elif operation == "divide":
                if b == 0:
                    return ToolResult.error_result("Division by zero is not allowed")
                result = a / b
            else:
                return ToolResult.error_result(f"Unknown operation: {operation}")
            
            return ToolResult.success_result(result)
        except Exception as e:
            return ToolResult.error_result(f"Error in calculator: {str(e)}")
    
    def get_schema(self) -> dict:
        """Get the schema for the calculator tool."""
        return {
            "parameters": {
                "operation": {
                    "type": "string",
                    "description": "The operation to perform (add, subtract, multiply, divide)",
                    "required": True
                },
                "a": {
                    "type": "number",
                    "description": "First number",
                    "required": True
                },
                "b": {
                    "type": "number",
                    "description": "Second number",
                    "required": True
                }
            },
            "returns": {
                "type": "number",
                "description": "Result of the calculation"
            }
        }

# Create the calculator tool
calculator = CalculatorTool()

## Understanding the Agent Core Behavior

The inconsistency occurs because of how the AgentCore class implements two separate methods:

1. `can_accomplish(task)` - Checks if a task can theoretically be accomplished with available tools
2. `process_message(message)` - Processes a message and generates a response

Let's look at both methods to understand why they're giving different results.

In [3]:
# Creating the agent with the calculator tool
config = AgentConfig(verbose=True, planning_enabled=True)
agent = AgentCore(config)
agent.register_tool(calculator)

# The task we want to check
task = "Can you calculate 42 divided by 6?"

# Let's evaluate if the task can be accomplished
assessment = agent.can_accomplish(task)
print(f"Task: {task}")
print(f"Assessment: {assessment['can_accomplish']}")
print(f"Reason: {assessment['reason']}")

# Now let's process the message
response = agent.process_message(task)
print(f"\nAgent Response: {response}")

2025-03-28 13:54:28,379 - agentic.core - INFO - Registering tool: calculator
2025-03-28 13:54:28,379 - agentic.core - INFO - Registering tool: calculator
2025-03-28 13:54:28,386 - agentic.core - INFO - Evaluating if task can be accomplished: Can you calculate 42 divided by 6?
2025-03-28 13:54:28,386 - agentic.core - INFO - Evaluating if task can be accomplished: Can you calculate 42 divided by 6?
2025-03-28 13:54:28,390 - agentic.planner - INFO - Generating plan for goal: Can you calculate 42 divided by 6?
2025-03-28 13:54:28,390 - agentic.planner - INFO - Generating plan for goal: Can you calculate 42 divided by 6?
2025-03-28 13:54:28,393 - agentic.planner - INFO - Generated plan with 3 steps
2025-03-28 13:54:28,393 - agentic.planner - INFO - Generated plan with 3 steps
2025-03-28 13:54:28,398 - agentic.core - INFO - Processing message from user: Can you calculate 42 divided by 6?
2025-03-28 13:54:28,379 - agentic.core - INFO - Registering tool: calculator
2025-03-28 13:54:28,386 - ag

Task: Can you calculate 42 divided by 6?
Assessment: True
Reason: The task can be accomplished with the current tools.


2025-03-28 13:54:28,410 - agentic.core - INFO - Executing plan for message: Can you calculate 42 divided by 6?
2025-03-28 13:54:28,419 - agentic.executor - INFO - Executing step: Analyze the request to understand the user's goal
2025-03-28 13:54:28,419 - agentic.executor - INFO - Executing step: Analyze the request to understand the user's goal
2025-03-28 13:54:28,419 - agentic.executor - INFO - Executing step: Analyze the request to understand the user's goal
2025-03-28 13:54:28,419 - agentic.executor - INFO - Executing step: Analyze the request to understand the user's goal
2025-03-28 13:54:28,427 - agentic.executor - INFO - Executing step: Determine if the goal can be accomplished with available tools
2025-03-28 13:54:28,427 - agentic.executor - INFO - Executing step: Determine if the goal can be accomplished with available tools
2025-03-28 13:54:28,430 - agentic.executor - INFO - Executing step: Inform the user about the capabilities that would be needed to accomplish the task
2025


Agent Response: I understand your request. However, I currently don't have the necessary tools to accomplish this task. As my capabilities develop, I'll be able to assist with more complex requests.


## Explaining the Discrepancy

The issue is in the implementation of the current AgentCore. Let's examine the key reasons:

### 1. The `can_accomplish` method

This method only checks if all the tools mentioned in the plan are available in the tool registry. If the agent's planner creates a plan that doesn't specifically name any tools (or only names tools that are available), it will report that the task can be accomplished.

```python
def can_accomplish(self, task: str) -> Dict[str, Any]:
    # Create a plan for the task
    plan = self.planning_engine.create_plan(goal=task, context=context)
    
    # Check if the plan can be executed
    tools_needed = []
    for step in plan.steps:
        if step.tool_name and not self.tool_registry.get_tool(step.tool_name):
            tools_needed.append(step.tool_name)
    
    if not tools_needed:
        return {
            'can_accomplish': True,
            'reason': 'The task can be accomplished with the current tools.'
        }
```

### 2. The `LLMPlanner` implementation

The current planner implementation is a placeholder that doesn't actually use a language model. Let's look at how it creates plans:

In [8]:
# Let's print the planner's create_plan method to understand its behavior
from agentic.core.agent import LLMPlanner, Plan

# Get the source code of the create_plan method
import inspect

print(inspect.getsource(LLMPlanner.create_plan))

    def create_plan(self, goal: str, context: Dict[str, Any]) -> Plan:
        """
        Create an execution plan for a given goal using a large language model.
        
        This implementation creates a plan based on the available tools and the goal.
        
        Args:
            goal: The goal to create a plan for
            context: Context information for planning
            
        Returns:
            An execution plan
        """
        # Create a new plan
        plan = Plan(goal=goal)
        
        # Get available tools for planning
        tools = context.get('available_tools', [])
        tool_descriptions = "\n".join([f"- {tool.name}: {tool.description}" for tool in tools])
        
        # Get conversation history for context
        conversation_history = context.get('conversation_history', '')
        
        # Construct prompt for the LLM
        prompt = f"""
You are an AI assistant that can break down tasks into specific steps.

GOAL: {goal}

AVAI

As we can see, the current planner implementation:

1. Creates a very generic plan that doesn't specify tool names for calculation tasks
2. The plan steps don't actually include the calculator tool name, even though the calculator is available

This means:
- `can_accomplish` returns `True` because no missing tools are detected in the plan
- But when the plan is executed, it doesn't actually use the calculator tool

### 3. Response Generation

Finally, the `_generate_success_response` method in the agent checks if any tools were actually used:

In [9]:
def _generate_success_response(self, plan: Plan) -> str:
    # If no tools were used in the plan, acknowledge that we can't do much yet
    tool_steps = [step for step in plan.steps if step.tool_name]
    
    if not tool_steps:
        return "I understand your request. However, I currently don't have the necessary tools " \
               "to accomplish this task. As my capabilities develop, I'll be able to assist " \
               "with more complex requests."
    else:
        # Summarize what was accomplished
        return f"I've completed the task: {plan.goal}\n\n" \
               f"I executed {len(tool_steps)} tool operations to accomplish this."

## Creating a Fix for this Issue

To fix this issue, we need to create an enhanced planner that actually recognizes calculation tasks and uses the calculator tool. Here's a simple implementation:

In [11]:
from agentic.core.planning import Plan, PlanStep, Planner
import re
from typing import Dict, Any

class EnhancedPlanner(Planner):
    """An enhanced planner that recognizes calculation tasks."""
    
    def __init__(self, agent_core):
        """Initialize the enhanced planner."""
        self.agent_core = agent_core
        self.logger = logging.getLogger('enhanced_planner')
    
    def create_plan(self, goal: str, context: Dict[str, Any]) -> Plan:
        """Create a plan based on the goal and available tools."""
        # Create a new plan
        plan = Plan(goal=goal)
        
        # Get available tools
        tools = context.get('available_tools', [])
        tool_map = {tool.name: tool for tool in tools}
        
        # Check if this is a calculation task
        if self._is_calculation_task(goal) and 'calculator' in tool_map:
            # Parse the calculation request
            operation, numbers = self._parse_calculation(goal)
            
            if operation and len(numbers) >= 2:
                # Add a step to understand the request
                plan.add_step(PlanStep(
                    description="Analyze the calculation request",
                    tool_name=None
                ))
                
                # Add a step to perform the calculation
                plan.add_step(PlanStep(
                    description=f"Calculate {numbers[0]} {operation} {numbers[1]}",
                    tool_name="calculator",
                    tool_args={
                        "operation": operation,
                        "a": float(numbers[0]),
                        "b": float(numbers[1])
                    }
                ))
                
                # Add a step to explain the result
                plan.add_step(PlanStep(
                    description="Provide the calculation result to the user",
                    tool_name=None
                ))
                
                return plan
        
        # For non-calculation tasks or if calculator isn't available, fall back to a generic plan
        plan.add_step(PlanStep(
            description="Analyze the request to understand the user's goal",
            tool_name=None
        ))
        
        # Check if we have any relevant tools
        available_tool_names = [tool.name for tool in tools]
        if not available_tool_names:
            plan.add_step(PlanStep(
                description="Inform the user that no tools are currently available",
                tool_name=None
            ))
        else:
            plan.add_step(PlanStep(
                description="Inform the user about the available capabilities",
                tool_name=None
            ))
        
        return plan
    
    def _is_calculation_task(self, text: str) -> bool:
        """Check if the text is asking for a calculation."""
        calculation_patterns = [
            r'calculate',
            r'compute',
            r'what is [\d\s\+\-\*\/]+',
            r'\d+\s*[\+\-\*\/]\s*\d+',
            r'(add|subtract|multiply|divide)'
        ]
        
        for pattern in calculation_patterns:
            if re.search(pattern, text.lower()):
                return True
        return False
    
    def _parse_calculation(self, text: str) -> tuple:
        """Parse a calculation request to extract operation and numbers."""
        # Extract numbers from the text
        numbers = re.findall(r'\d+', text)
        
        # Determine the operation
        operation = None
        if 'add' in text.lower() or '+' in text:
            operation = 'add'
        elif 'subtract' in text.lower() or '-' in text:
            operation = 'subtract'
        elif 'multiply' in text.lower() or '*' in text or 'times' in text.lower():
            operation = 'multiply'
        elif 'divide' in text.lower() or '/' in text:
            operation = 'divide'
        
        return operation, numbers

## Testing the Enhanced Planner

Now let's create a new agent with our enhanced planner and try the same task:

In [12]:
from agentic.core.agent import AgentExecutor, PlanningEngine

# Create a new agent with the enhanced planner
new_config = AgentConfig(verbose=True, planning_enabled=True)
new_agent = AgentCore(new_config)
new_agent.register_tool(calculator)

# Replace the default planner with our enhanced planner
new_agent.planner = EnhancedPlanner(new_agent)
new_agent.planning_engine = PlanningEngine(new_agent.planner, new_agent.executor)

# The same task as before
task = "Can you calculate 42 divided by 6?"

# Evaluate if the task can be accomplished
assessment = new_agent.can_accomplish(task)
print(f"Task: {task}")
print(f"Assessment: {assessment['can_accomplish']}")
print(f"Reason: {assessment['reason']}")

# Process the message
response = new_agent.process_message(task)
print(f"\nEnhanced Agent Response: {response}")

2025-03-28 13:58:24,859 - agentic.core - INFO - Registering tool: calculator
2025-03-28 13:58:24,859 - agentic.core - INFO - Registering tool: calculator
2025-03-28 13:58:24,861 - agentic.core - INFO - Evaluating if task can be accomplished: Can you calculate 42 divided by 6?
2025-03-28 13:58:24,861 - agentic.core - INFO - Evaluating if task can be accomplished: Can you calculate 42 divided by 6?
2025-03-28 13:58:24,865 - agentic.core - INFO - Processing message from user: Can you calculate 42 divided by 6?
2025-03-28 13:58:24,865 - agentic.core - INFO - Processing message from user: Can you calculate 42 divided by 6?
2025-03-28 13:58:24,867 - agentic.core - INFO - Executing plan for message: Can you calculate 42 divided by 6?
2025-03-28 13:58:24,867 - agentic.core - INFO - Executing plan for message: Can you calculate 42 divided by 6?
2025-03-28 13:58:24,868 - agentic.executor - INFO - Executing step: Analyze the calculation request
2025-03-28 13:58:24,868 - agentic.executor - INFO - 

Task: Can you calculate 42 divided by 6?
Assessment: True
Reason: The task can be accomplished with the current tools.

Enhanced Agent Response: I've completed the task: Can you calculate 42 divided by 6?

I executed 1 tool operations to accomplish this.


## Conclusion

This example demonstrates why there was a discrepancy in the original agent behavior:

1. The default planner doesn't create plans that actually use the calculator tool, even for calculation tasks
2. The `can_accomplish` method only checks if requested tools exist, not if the planner is smart enough to use them
3. The response generation notices that no tools were actually used in the plan and provides a fallback response

Our enhanced planner fixes this by actually recognizing calculation tasks and explicitly including the calculator tool in the plan.

In a full implementation with a real LLM-based planner, the LLM would be expected to recognize that the calculator tool should be used for this task and create an appropriate plan that specifies the tool.